In [35]:
# Use this block of code when working on Google Colab in order to save the files in Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

#path = "drive/MyDrive/Thesis/"

### Install and import the necesary libraries

In [36]:
# Installation of libraries for Google Colab
# %pip install osmnx
# %pip install vt2geojson
# %pip install mercantile

In [37]:
# Libraries for working with maps and geospatial data
from vt2geojson.tools import vt_bytes_to_geojson
from shapely.geometry import Point
from scipy.spatial import cKDTree
import geopandas as gpd
import osmnx as ox
import mercantile

# Libraries for working with concurrency and file manipulation
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pandas as pd
import requests

### Get the roads of the city

We use OSMnx to retrieve the road network of the selected city and save the result as a GeoPackage file.

In [38]:
def get_road_network(city):
    # Get the road network graph using OpenStreetMap data
    # 'network_type' argument is set to 'drive' to get the road network suitable for driving
    # 'simplify' argument is set to 'True' to simplify the road network
    G = ox.graph_from_place(city, network_type="drive", simplify=True)

    # Project the graph from latitude-longitude coordinates to a local projection (in meters)
    G_proj = ox.project_graph(G)

    # Convert the projected graph to a GeoDataFrame
    _, edges = ox.graph_to_gdfs(G_proj)    

    return edges

### Get the road edges and select the points

The previously saved GeoPackage file with the road network data is read to then create a list of points over the road map with a defined distance between them.

In [39]:
# Get a list of points over the road map with a N distance between them
def select_points_on_road_network(roads, distance=50):
    # Initialize a list to store the points
    points = []

    # Loop through each road in the road network graph
    for road in roads.geometry:
        # Calculate the total length of the road
        road_length = road.length

        # Start at the beginning of the road
        current_position = 0

        # Loop through the road, adding points every 50 meters
        while current_position < road_length:
            # Get the point on the road at the current position
            current_point = road.interpolate(current_position)

            # Add the curent point to the list of points
            points.append(current_point)

            # Increment the position by the desired distance
            current_position += distance
    
    # Convert the list of points to a GeoDataFrame
    gdf_points = gpd.GeoDataFrame(geometry=points)

    # Set the same CRS as the road dataframes for the points dataframe
    gdf_points.set_crs(roads.crs, inplace=True)

    return gdf_points

### Get the features on the selected points on the roads

Here, we group the points by their corresponding tiles and download the tiles and extract the features for each group. Then, the features are mateched to each point and the results are saved as a GeoPackage file.

In [40]:
# This function extracts the features for a given tile
def get_features_for_tile(tile, access_token):
    tile_url = f"https://tiles.mapillary.com/maps/vtp/mly1_public/2/{tile.z}/{tile.x}/{tile.y}?access_token={access_token}"
    response = requests.get(tile_url)
    result = vt_bytes_to_geojson(response.content, tile.x, tile.y, tile.z, layer="image")
    return [tile, result]

In [41]:
def get_features_on_points(points, access_token, zoom=14):
    # Transform the coordinate reference system to EPSG 4326
    points.to_crs(epsg=4326, inplace=True)

    # Add a new column to gdf_points that contains the tile coordinates for each point
    points['tile'] = [mercantile.tile(x, y, zoom) for x, y in zip(points.geometry.x, points.geometry.y)]

    # Group the points by their corresponding tiles
    groups = points.groupby('tile')

    # Download the tiles and extract the features for each group
    features = []
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []

        for tile, _ in groups:
            futures.append(executor.submit(get_features_for_tile, tile, access_token))
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Downloading tiles"):
            result = future.result()
            features.append(result)

    pd_features = pd.DataFrame(features, columns=["tile", "features"])

    # Compute distances between each feature and all the points in gdf_points
    feature_points = pd.DataFrame(
        [(Point(f["geometry"]["coordinates"]), f) for row in pd_features["features"] for f in row["features"]],
        columns=["geometry", "feature"]
    )
    feature_tree = cKDTree(feature_points["geometry"].apply(lambda p: [p.x, p.y]).tolist())
    _, indices = feature_tree.query(points["geometry"].apply(lambda p: [p.x, p.y]).tolist())

    # Select the closest feature for each point
    points["feature"] = feature_points.loc[indices, "feature"].tolist()

    # Convert results to geodataframe
    points['tile'] = points['tile'].astype(str)
    
    return points

### Check the code with Kampala, Uganda

In [42]:
# Get the roadnetwork of a specific city using OpenStreetMap data
city = "Kampala, Uganda"

# Set access token for mapillary
access_token = "MLY|6267906093323631|fba37c53726a386c951323ee5b9874bf"
path = ""

road = get_road_network(city)
points = select_points_on_road_network(road)
features = get_features_on_points(points, access_token)

In [43]:
path_to_file="{}results/{}/data/points_with_features.gpkg".format(path, city, city)
features.to_file(path_to_file, driver="GPKG", crs=features.crs)

,geometry,tile,feature
0,POINT (32.59714 0.33946),"Tile(x=9675, y=8176, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."
1,POINT (32.59714 0.33946),"Tile(x=9675, y=8176, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."
2,POINT (32.59753 0.33924),"Tile(x=9675, y=8176, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."
3,POINT (32.59793 0.33903),"Tile(x=9675, y=8176, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."
4,POINT (32.59832 0.33881),"Tile(x=9675, y=8176, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."
...,...,...,...
95612,POINT (32.58851 0.36666),"Tile(x=9675, y=8175, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."
95613,POINT (32.58851 0.36666),"Tile(x=9675, y=8175, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."
95614,POINT (32.58895 0.36667),"Tile(x=9675, y=8175, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."
95615,POINT (32.58849 0.36636),"Tile(x=9675, y=8175, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."


### Use the code for Mexico City, Mexico

In [44]:
# Get the roadnetwork of a specific city using OpenStreetMap data
city = "Mexico City, Mexico"

# Set access token for mapillary
access_token = "MLY|6267906093323631|fba37c53726a386c951323ee5b9874bf"
path = ""

road = get_road_network(city)
points = select_points_on_road_network(road)
features = get_features_on_points(points, access_token)

In [45]:
path_to_file="{}results/{}/data/points_with_features.gpkg".format(path, city, city)
features.to_file(path_to_file, driver="GPKG", crs=features.crs)

In [46]:
features

,geometry,tile,feature
0,POINT (-99.13410 19.41806),"Tile(x=3680, y=7290, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."
1,POINT (-99.13404 19.41851),"Tile(x=3680, y=7290, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."
2,POINT (-99.13397 19.41895),"Tile(x=3680, y=7290, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."
3,POINT (-99.13392 19.41934),"Tile(x=3680, y=7290, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."
4,POINT (-99.13344 19.41928),"Tile(x=3680, y=7290, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."
...,...,...,...
642830,POINT (-99.19595 19.21867),"Tile(x=3677, y=7300, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."
642831,POINT (-99.19679 19.21870),"Tile(x=3677, y=7300, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."
642832,POINT (-99.19687 19.21914),"Tile(x=3677, y=7300, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."
642833,POINT (-99.19794 19.21595),"Tile(x=3677, y=7300, z=14)","{'type': 'Feature', 'geometry': {'type': 'Poin..."
